In [1]:
import pandas as pd
import numpy as np
import csv
import re
from tqdm import tqdm

In [2]:
data = pd.read_csv('cat_train_v2.csv')

data = data[['name','product']]
print('資料總筆數:', len(data))
print('品類總數:', len(data['product'].unique()))

資料總筆數: 92306
品類總數: 272


In [3]:
# 全形轉半形
def strQ2B(ustring):
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:                            # 全形空格直接轉換
            inside_code = 32
        elif 65281 <= inside_code <= 65374:                # 全形字元（除空格）根據關係轉化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

In [4]:
# 刪除名稱單純為數字
num_data = data[(data['name'].str.isnumeric())]
data = data[(data['name'].str.isnumeric() == 0)].reset_index()
print('刪除名稱為數字的資料總筆數:', len(data))

刪除名稱為數字的資料總筆數: 92299


In [5]:
# 大寫轉小寫
data['name'] = data['name'].str.lower()

for i in tqdm(range(len(data))):
    try:
        data['name'][i] = strQ2B(data['name'][i])
    except:
        pass
data

  0%|                                                 | 0/92299 [00:00<?, ?it/s]/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_67753/1607984465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['name'][i] = strQ2B(data['name'][i])
100%|████████████████████████████████████| 92299/92299 [04:03<00:00, 378.83it/s]


,index,name,product
0,0,r視舒坦單支裝人工淚液點眼液0.5ml 24支入,人工淚液
1,1,優麗舒點眼液15ml,人工淚液
2,2,(r)視舒坦單支裝人工,人工淚液
3,3,rohto乾眼淚液,人工淚液
4,4,r參天柔潤人工淚液點眼液 5ml 4入,人工淚液
...,...,...,...
92294,92301,mdmmd微醺心機漸層腮紅-玫瑰瑪格麗特,腮紅
92295,92302,mdmmd微醺心機漸層腮紅-橙酒,腮紅
92296,92303,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,腮紅
92297,92304,palladio柔光幻彩貝殼腮紅微醺蜜桃,腮紅


暫存檔

In [6]:
presave = data.copy(deep=True)

In [7]:
data = presave.copy(deep=True)
data = data[['name','product']]

# 清理多餘字詞

In [140]:
def clean_mssy(text):
    # 中文量詞
    remove_measure = re.sub(r'''(x|\*)?([0-9]+( +)?(入|支|包|片|瓶|罐|盒|杯|(顆)?袋)+裝|[0-9]+\.[0-9]+包|[0-9]+\.[0-9]+片|[0-9]+\.[0-9]+瓶|[0-9]+\.[0-9]+號|[0-9]+\.[0-9]+罐|[0-9]+\.[0-9]+盒|[0-9]+\.[0-9]+組|[0-9]+\.[0-9]+杯|[0-9]+\.[0-9]+公分|[0-9]+\.[0-9]+公克|[0-9]+\.[0-9]+公斤|[0-9]+\.[0-9]+公升|[0-9]+\.[0-9]+升|[0-9]+\.[0-9]+人份|[0-9]+\.[0-9]+支|[0-9]+\.[0-9]+元|[0-9]+\.[0-9]+支入|[0-9]+\.[0-9]+毫升|[0-9]+\.[0-9]+毫克|[0-9]+\.[0-9]+件|[0-9]+\.[0-9]+日|[0-9]+\.[0-9]+克|[0-9]+\.[0-9]+袋|[0-9]+\.[0-9]+斤|[0-9]+\.[0-9]+磅|[0-9]+\.[0-9]+箱|[0-9]+\.[0-9]+裝|[0-9]+\.[0-9]+顆|[0-9]+\.[0-9]+粒|[0-9]+\.[0-9]+錠|[0-9]+\.[0-9]+桶|[0-9]+\.[0-9]+塊|[0-9]+\.[0-9]+束|[0-9]+\.[0-9]+條|[0-9]+\.[0-9]+吋|[0-9]+\.[0-9]+抽|[0-9]+\.[0-9]+個|[0-9]+\.[0-9]+枚|[0-9]+\.[0-9]+張|[0-9]+\.[0-9]+串|[0-9]+\.[0-9]+捲|[0-9]+\.[0-9]+雙|[0-9]+\.[0-9]+加侖|[0-9]+\.[0-9]+公尺|[0-9]+\.[0-9]+頁|[0-9]+\.[0-9]+歲|[0-9]+\.[0-9]+折|[0-9]+\.[0-9]+款|[0-9]+\.[0-9]+本|[0-9]+\.[0-9]+筒|[0-9]+\.[0-9]+入|下殺[0-9]+\.[0-9]|[0-9]+支入|[0-9]+公克|[0-9]+包|[0-9]+片|[0-9]+瓶|[0-9]+號|[0-9]+罐|[0-9]+盒|[0-9]+組|[0-9]+公分|[0-9]+( )?公斤|[0-9]+入(組|盒)|[0-9]+公升|[0-9]+( )?份入|[0-9]+毫升|[0-9]+毫克|[0-9]+克|[0-9]+袋|[0-9]+( )?磅|[0-9]+箱|[0-9]+裝|[0-9]+顆((袋|盒)裝)?|[0-9]+粒|[0-9]+錠|[0-9]+桶|[0-9]+塊|[0-9]+束|[0-9]+條|[0-9]+吋|[0-9]+抽|[0-9]+個|[0-9]+枚(入)?|[0-9]+張|[0-9]+串|[0-9]+捲|[0-9]+雙|[0-9]+加侖|[0-9]+公尺|[0-9]+頁|[0-9]+本|[0-9]+筒|[0-9]+(\+[0-9])?( +)?入|[0-9]+件(組)?|[0-9]+日|[0-9]+升|[0-9]+人份|[0-9]+碗(入)?|[0-9]+杯|[0-9]+(-[0-9]+)?歲|[0-9]+折|[0-9]+款|[0-9]+支|[0-9]+元|[0-9]+( )?吋|[0-9]+斤|[0-9]+坪|下殺[0-9])''',"",str(text))
    
    # 英文量詞
    remove_egmeasure = re.sub(r'''[0-9]+\.[0-9]+ml|[0-9]+\.[0-9]+l|[0-9]+\.[0-9]+oz|[0-9]+\.[0-9]+%|[0-9]+\.[0-9]+g|[0-9]+\.[0-9]+kg|[0-9]+\.[0-9]+cm|[0-9]+\.[0-9]+mm|[0-9]+\.[0-9]+mg|[0-9]+\.[0-9]+m|[0-9]+\.[0-9]+cc|[0-9]+\.[0-9]+c.c.|[0-9]+\.[0-9]+CC|[0-9]+\.[0-9]+gm|[0-9]+\.[0-9]+pg|$[0-9]+\.[0-9]+|[0-9]+ml|[0-9]+ ml|[0-9]+l|[0-9]+oz|[0-9]+g|[0-9]+ g|[0-9]+kg|([0-9]+\*)?[0-9]+cm|[0-9]+mm|[0-9]+mg|[0-9]+m|[0-9]+cc|[0-9]+c.c.|[0-9]+gm|[0-9]+pg|[0-9]+%|$[0-9]''', "", remove_measure)
    
    # 複合詞
    remove_both = re.sub(r'''^[0-9]{6,15}|[0-9]+@|([0-9]+)?/+(包|裝|盒|kg|組|支|瓶|袋|罐|箱|顆)|\d{4}/\d{1,2}/\d{1,2}|\d{4}/\d{1,2}|\d{4}.\d{1,2}.\d{1,2}|\d{4}.\d{1,2}|長[0-9]+(\.[0-9]+)?x寬[0-9]+(\.[0-9]+)?x高([0-9]+\.[0-9]+)?''', "", remove_egmeasure)
    
    # 行銷詞
    remove_buss = re.sub(r'''買一送一|買+[0-9]+送+[0-9]|[0-9]+送+[0-9]|現貨|蝦皮24h|24h|24 h|代收代付\-|代收代付|任選|特惠組|(保存|最低)(期限|效期至|效期至)|效期\d{4}|效期|期限至|期限|效期至|下單|出貨|官方直營|免運(費)?|型號(:)?\w+''', "", remove_both)
    
    # 特殊符號與自訂
    remove_symbol = re.sub(r'''\d\+\d|[0-9]+\.[0-9]+$|[0-9]+\$|\d{1,3}$|^\s+|\s+$|\*[0-9]+[a-z]+|\~[a-z]+\~|^[a-z]\-|\![a-z+]\-|(x|\*)[0-9]|(x|\*)+[0-9]+\.[0-9]|[!;:"'{}'.,/?\@#<>&※〔〕【】「」『』\[\]《》〈〉«»、∥=→↑。\│±()+\-^~_\*\$|♦▼▶★☆❤㊣╳╱♠⚡️◆✱▲↘✬✪™∮☛●◤◢≡⭐️✴️✨☀◖◗®]''', "", remove_buss) 
    
    return remove_symbol

In [141]:
data['clean_name'] = ''
data['clean_name'] = data.name.apply(clean_mssy)

# data.loc[data['clean_name'] == '']
data

,name,product,clean_name
0,r視舒坦單支裝人工淚液點眼液0.5ml 24支入,人工淚液,r視舒坦單支裝人工淚液點眼液
1,優麗舒點眼液15ml,人工淚液,優麗舒點眼液
2,(r)視舒坦單支裝人工,人工淚液,r視舒坦單支裝人工
3,rohto乾眼淚液,人工淚液,rohto乾眼淚液
4,r參天柔潤人工淚液點眼液 5ml 4入,人工淚液,r參天柔潤人工淚液點眼液
...,...,...,...
92294,mdmmd微醺心機漸層腮紅-玫瑰瑪格麗特,腮紅,mdmmd微醺心機漸層腮紅玫瑰瑪格麗特
92295,mdmmd微醺心機漸層腮紅-橙酒,腮紅,mdmmd微醺心機漸層腮紅橙酒
92296,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,腮紅,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅
92297,palladio柔光幻彩貝殼腮紅微醺蜜桃,腮紅,palladio柔光幻彩貝殼腮紅微醺蜜桃


## 轉出還有數字的檔案

In [205]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [210]:
check_num = data.copy(deep=True)
check_num = check_num[['name','clean_name']]
check_num['tmp'] = ''
check_num['tmp'] = data.clean_name.apply(has_numbers)
check_num = check_num.loc[check_num['tmp'] == 1].drop('tmp', axis=1).reset_index()
check_num

,index,name,clean_name
0,13,(r)e視界-50點眼液,re視界50點眼液
1,18,!b-e視界-50點眼液,e視界50點眼液
2,69,r新一點靈b12滴眼液15ml-薄荷加量,r新一點靈b12滴眼液薄荷加量
3,83,海滴人工淚液 1.4 10ml(otc),海滴人工淚液 14 otc
4,94,目立康護理液250ml*2+目立康去蛋白*1+愛力根潤濕液1瓶~贈去蛋白盒*1.8字水盒x1,目立康護理液目立康去蛋白愛力根潤濕液贈去蛋白盒8字水盒
...,...,...,...
22647,92291,【304不鏽鋼-烈酒酒嘴】酒嘴 油嘴 烈酒酒嘴 不銹鋼酒嘴 調酒器 調酒嘴 倒酒器 注酒器 ...,304不鏽鋼烈酒酒嘴酒嘴 油嘴 烈酒酒嘴 不銹鋼酒嘴 調酒器 調酒嘴 倒酒器 注酒器 導流器...
22648,92292,mkup不怕水粉霧持久唇釉-01微醺玫瑰粉2.5g,mkup不怕水粉霧持久唇釉01微醺玫瑰粉
22649,92293,mkup不怕水粉霧持久唇釉-01微醺玫瑰粉25g,mkup不怕水粉霧持久唇釉01微醺玫瑰粉
22650,92296,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅


In [5]:
df = pd.read_csv('./temp/contain_num_cat.csv')
df

,Unnamed: 0,index,name,clean_name
0,0,13,(r)e視界-50點眼液,re視界50點眼液
1,1,18,!b-e視界-50點眼液,e視界50點眼液
2,2,69,r新一點靈b12滴眼液15ml-薄荷加量,r新一點靈b12滴眼液薄荷加量
3,3,83,海滴人工淚液 1.4 10ml(otc),海滴人工淚液 14 otc
4,4,94,目立康護理液250ml*2+目立康去蛋白*1+愛力根潤濕液1瓶~贈去蛋白盒*1.8字水盒x1,目立康護理液目立康去蛋白愛力根潤濕液贈去蛋白盒8字水盒
...,...,...,...,...
22647,22647,92291,【304不鏽鋼-烈酒酒嘴】酒嘴 油嘴 烈酒酒嘴 不銹鋼酒嘴 調酒器 調酒嘴 倒酒器 注酒器 ...,304不鏽鋼烈酒酒嘴酒嘴 油嘴 烈酒酒嘴 不銹鋼酒嘴 調酒器 調酒嘴 倒酒器 注酒器 導流器...
22648,22648,92292,mkup不怕水粉霧持久唇釉-01微醺玫瑰粉2.5g,mkup不怕水粉霧持久唇釉01微醺玫瑰粉
22649,22649,92293,mkup不怕水粉霧持久唇釉-01微醺玫瑰粉25g,mkup不怕水粉霧持久唇釉01微醺玫瑰粉
22650,22650,92296,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅


In [6]:
df.to_csv('./temp/contain_num_cat.csv', encoding="utf-8")

In [218]:
data = data[['name','product','clean_name']]
data.to_csv('./temp/cleaned.csv')
data

,name,product,clean_name
0,r視舒坦單支裝人工淚液點眼液0.5ml 24支入,人工淚液,r視舒坦單支裝人工淚液點眼液
1,優麗舒點眼液15ml,人工淚液,優麗舒點眼液
2,(r)視舒坦單支裝人工,人工淚液,r視舒坦單支裝人工
3,rohto乾眼淚液,人工淚液,rohto乾眼淚液
4,r參天柔潤人工淚液點眼液 5ml 4入,人工淚液,r參天柔潤人工淚液點眼液
...,...,...,...
92294,mdmmd微醺心機漸層腮紅-玫瑰瑪格麗特,腮紅,mdmmd微醺心機漸層腮紅玫瑰瑪格麗特
92295,mdmmd微醺心機漸層腮紅-橙酒,腮紅,mdmmd微醺心機漸層腮紅橙酒
92296,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,腮紅,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅
92297,palladio柔光幻彩貝殼腮紅微醺蜜桃,腮紅,palladio柔光幻彩貝殼腮紅微醺蜜桃


# 多標籤處理

1. 多標籤標記

In [142]:
def multitag(d):
    d['tag'] = ''
    
    for i in tqdm(range(len(d))):
        string = d['product'][i]
        s = string.split(';')
        d['tag'][i] = len(s)

In [143]:
multi_tags = data.copy(deep=True)

multitag(multi_tags)

100%|██████████████████████████████████| 92299/92299 [00:07<00:00, 12300.26it/s]


In [144]:
multi_tags['tag'].value_counts()

1    91912
2      374
3       12
4        1
Name: tag, dtype: int64

2. 刪除多標籤

In [146]:
removed_tag = multi_tags.copy(deep=True)
removed_tag.drop(removed_tag.loc[removed_tag['tag'] != 1].index, inplace=True)
removed_tag = removed_tag[['clean_name','product']]
removed_tag

,clean_name,product
0,r視舒坦單支裝人工淚液點眼液,人工淚液
1,優麗舒點眼液,人工淚液
2,r視舒坦單支裝人工,人工淚液
3,rohto乾眼淚液,人工淚液
4,r參天柔潤人工淚液點眼液,人工淚液
...,...,...
92294,mdmmd微醺心機漸層腮紅玫瑰瑪格麗特,腮紅
92295,mdmmd微醺心機漸層腮紅橙酒,腮紅
92296,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,腮紅
92297,palladio柔光幻彩貝殼腮紅微醺蜜桃,腮紅


In [148]:
removed_tag.columns = ['name','product']
removed_tag.to_csv('one_tag.csv')

3. 只有多標籤

In [147]:
mul_tag = multi_tags.copy(deep=True)
mul_tag.drop(mul_tag.loc[mul_tag['tag'] == 1].index, inplace=True)
mul_tag = mul_tag[['clean_name','product']]
mul_tag

,clean_name,product
9182,原萃烏龍茶,即飲烏龍茶;即飲無糖茶
9183,冷萃金萱烏龍茶,即飲烏龍茶;即飲無糖茶
9184,原萃冷萃 金萱烏龍茶,即飲烏龍茶;即飲無糖茶
9185,原萃冷萃烏龍茶,即飲烏龍茶;即飲無糖茶
9186,原萃烏龍茶,即飲烏龍茶;即飲無糖茶
...,...,...
91672,保證bb霜 隔離修容bb美顏霜 粉底液 遮瑕 修容,BB霜;粉底液
91673,庶務客 pony effect 遮瑕氣墊粉餅 粉底 神防護 氣墊 bb霜 氣墊粉餅,BB霜;粉餅
91736,kose 高絲雪肌精透亮煥白cc霜雪肌精淨透潔顏油n 正統公司貨,CC霜;卸妝油
91737,drcink 達特聖克女神絲絨柔焦組cc霜花蜜露潔面露,CC霜;洗面乳


In [149]:
mul_tag.columns = ['name','product']
mul_tag.to_csv('more_tag.csv')

In [151]:
mul_tag = pd.read_csv('more_tag.csv')
mul_tag = mul_tag[['name', 'product']]

In [152]:
testfinal = pd.concat([removed_tag,mul_tag], ignore_index=True)
testfinal

,name,product
0,r視舒坦單支裝人工淚液點眼液,人工淚液
1,優麗舒點眼液,人工淚液
2,r視舒坦單支裝人工,人工淚液
3,rohto乾眼淚液,人工淚液
4,r參天柔潤人工淚液點眼液,人工淚液
...,...,...
92294,保證bb霜 隔離修容bb美顏霜 粉底液 遮瑕 修容,BB霜
92295,庶務客 pony effect 遮瑕氣墊粉餅 粉底 神防護 氣墊 bb霜 氣墊粉餅,粉餅
92296,kose 高絲雪肌精透亮煥白cc霜雪肌精淨透潔顏油n 正統公司貨,CC霜
92297,drcink 達特聖克女神絲絨柔焦組cc霜花蜜露潔面露,CC霜


In [153]:
testfinal.to_csv('cleaned_train.csv')

## 整理 Testing Data

In [13]:
testdata = pd.read_csv('cat_test_v2(question).csv')
testdata = testdata[['name','product']]
testdata['product'] = ''

In [14]:
testdata

,name,product
0,【施巴】嬌顏卸妝乳HA400ml,
1,艾瑪絲舒敏平衡洗髮精40,
2,【BB LOVE】高質感天絲TENCEL舖棉兩用兒童睡袋(多款可選),
3,Lovita愛維他-緩釋型維生素C 1000mg(60錠),
4,111262674618 CM 12公分長效Rifle風扇 1200轉,
...,...,...
108846,歐舒丹~橙花蘭花護手霜30ml,
108847,高露潔 360度抗敏專家牙刷(1入),
108848,高露潔專業型超小刷頭密植軟毛牙刷6支裝【康是美】,
108849,熊寶貝 沁藍海洋香柔軟衣物護衣精2+6組 (3.2Lx2+1.84Lx6包),


In [15]:
# 大寫轉小寫
testdata['name'] = testdata['name'].str.lower()

for i in tqdm(range(len(testdata))):
    try:
        testdata['name'][i] = strQ2B(testdata['name'][i])
    except:
        pass

100%|██████████████████████████████| 108851/108851 [00:00<00:00, 1984595.39it/s]


In [16]:
def cleanf(text):
    
    if not text.isnumeric():
        # 中文量詞
        remove_measure = re.sub(r'''(x|\*)?([0-9]+( +)?(入|支|包|片|瓶|罐|盒|杯|(顆)?袋)+裝|[0-9]+\.[0-9]+包|[0-9]+\.[0-9]+片|[0-9]+\.[0-9]+瓶|[0-9]+\.[0-9]+號|[0-9]+\.[0-9]+罐|[0-9]+\.[0-9]+盒|[0-9]+\.[0-9]+組|[0-9]+\.[0-9]+杯|[0-9]+\.[0-9]+公分|[0-9]+\.[0-9]+公克|[0-9]+\.[0-9]+公斤|[0-9]+\.[0-9]+公升|[0-9]+\.[0-9]+升|[0-9]+\.[0-9]+人份|[0-9]+\.[0-9]+支|[0-9]+\.[0-9]+元|[0-9]+\.[0-9]+支入|[0-9]+\.[0-9]+毫升|[0-9]+\.[0-9]+毫克|[0-9]+\.[0-9]+件|[0-9]+\.[0-9]+日|[0-9]+\.[0-9]+克|[0-9]+\.[0-9]+袋|[0-9]+\.[0-9]+斤|[0-9]+\.[0-9]+磅|[0-9]+\.[0-9]+箱|[0-9]+\.[0-9]+裝|[0-9]+\.[0-9]+顆|[0-9]+\.[0-9]+粒|[0-9]+\.[0-9]+錠|[0-9]+\.[0-9]+桶|[0-9]+\.[0-9]+塊|[0-9]+\.[0-9]+束|[0-9]+\.[0-9]+條|[0-9]+\.[0-9]+吋|[0-9]+\.[0-9]+抽|[0-9]+\.[0-9]+個|[0-9]+\.[0-9]+枚|[0-9]+\.[0-9]+張|[0-9]+\.[0-9]+串|[0-9]+\.[0-9]+捲|[0-9]+\.[0-9]+雙|[0-9]+\.[0-9]+加侖|[0-9]+\.[0-9]+公尺|[0-9]+\.[0-9]+頁|[0-9]+\.[0-9]+歲|[0-9]+\.[0-9]+折|[0-9]+\.[0-9]+款|[0-9]+\.[0-9]+本|[0-9]+\.[0-9]+筒|[0-9]+\.[0-9]+入|下殺[0-9]+\.[0-9]|[0-9]+支入|[0-9]+公克|[0-9]+包|[0-9]+片|[0-9]+瓶|[0-9]+號|[0-9]+罐|[0-9]+盒|[0-9]+組|[0-9]+公分|[0-9]+( )?公斤|[0-9]+入(組|盒)|[0-9]+公升|[0-9]+( )?份入|[0-9]+毫升|[0-9]+毫克|[0-9]+克|[0-9]+袋|[0-9]+( )?磅|[0-9]+箱|[0-9]+裝|[0-9]+顆((袋|盒)裝)?|[0-9]+粒|[0-9]+錠|[0-9]+桶|[0-9]+塊|[0-9]+束|[0-9]+條|[0-9]+吋|[0-9]+抽|[0-9]+個|[0-9]+枚(入)?|[0-9]+張|[0-9]+串|[0-9]+捲|[0-9]+雙|[0-9]+加侖|[0-9]+公尺|[0-9]+頁|[0-9]+本|[0-9]+筒|[0-9]+(\+[0-9])?( +)?入|[0-9]+件(組)?|[0-9]+日|[0-9]+升|[0-9]+人份|[0-9]+碗(入)?|[0-9]+杯|[0-9]+(-[0-9]+)?歲|[0-9]+折|[0-9]+款|[0-9]+支|[0-9]+元|[0-9]+( )?吋|[0-9]+斤|[0-9]+坪|下殺[0-9])''',"",str(text))

        # 英文量詞
        remove_egmeasure = re.sub(r'''[0-9]+\.[0-9]+ml|[0-9]+\.[0-9]+l|[0-9]+\.[0-9]+oz|[0-9]+\.[0-9]+%|[0-9]+\.[0-9]+g|[0-9]+\.[0-9]+kg|[0-9]+\.[0-9]+cm|[0-9]+\.[0-9]+mm|[0-9]+\.[0-9]+mg|[0-9]+\.[0-9]+m|[0-9]+\.[0-9]+cc|[0-9]+\.[0-9]+c.c.|[0-9]+\.[0-9]+CC|[0-9]+\.[0-9]+gm|[0-9]+\.[0-9]+pg|$[0-9]+\.[0-9]+|[0-9]+ml|[0-9]+ ml|[0-9]+l|[0-9]+oz|[0-9]+g|[0-9]+ g|[0-9]+kg|([0-9]+\*)?[0-9]+cm|[0-9]+mm|[0-9]+mg|[0-9]+m|[0-9]+cc|[0-9]+c.c.|[0-9]+gm|[0-9]+pg|[0-9]+%|$[0-9]''', "", remove_measure)

        # 複合詞
        remove_both = re.sub(r'''^[0-9]{6,15}|[0-9]+@|([0-9]+)?/+(包|裝|盒|kg|組|支|瓶|袋|罐|箱|顆)|\d{4}/\d{1,2}/\d{1,2}|\d{4}/\d{1,2}|\d{4}.\d{1,2}.\d{1,2}|\d{4}.\d{1,2}|長[0-9]+(\.[0-9]+)?x寬[0-9]+(\.[0-9]+)?x高([0-9]+\.[0-9]+)?''', "", remove_egmeasure)

        # 行銷詞
        remove_buss = re.sub(r'''買一送一|買+[0-9]+送+[0-9]|[0-9]+送+[0-9]|現貨|蝦皮24h|24h|24 h|代收代付\-|代收代付|任選|特惠組|(保存|最低)(期限|效期至|效期至)|效期\d{4}|效期|期限至|期限|效期至|下單|出貨|官方直營|免運(費)?|型號(:)?\w+''', "", remove_both)

        # 特殊符號與自訂
        remove_symbol = re.sub(r'''\d\+\d|[0-9]+\.[0-9]+$|[0-9]+\$|\d{1,3}$|^\s+|\s+$|\*[0-9]+[a-z]+|\~[a-z]+\~|^[a-z]\-|\![a-z+]\-|(x|\*)[0-9]|(x|\*)+[0-9]+\.[0-9]|[!;:"'{}'.,/?\@#<>&※〔〕【】「」『』\[\]《》〈〉«»、∥=→↑。\│±()+\-^~_\*\$|♦▼▶★☆❤㊣╳╱♠⚡️◆✱▲↘✬✪™∮☛●◤◢≡⭐️✴️✨☀◖◗®]''', "", remove_buss) 
    else:
        remove_symbol = text
    return remove_symbol

In [17]:
testdata['clean_name'] = ''
testdata['clean_name'] = testdata.name.apply(cleanf)

In [18]:
testdata

,name,product,clean_name
0,【施巴】嬌顏卸妝乳ha400ml,,施巴嬌顏卸妝乳ha
1,艾瑪絲舒敏平衡洗髮精40,,艾瑪絲舒敏平衡洗髮精
2,【bb love】高質感天絲tencel舖棉兩用兒童睡袋(多款可選),,bb love高質感天絲tencel舖棉兩用兒童睡袋多款可選
3,lovita愛維他-緩釋型維生素c 1000mg(60錠),,lovita愛維他緩釋型維生素c
4,111262674618 cm 12公分長效rifle風扇 1200轉,,cm 長效rifle風扇 1200轉
...,...,...,...
108846,歐舒丹~橙花蘭花護手霜30ml,,歐舒丹橙花蘭花護手霜
108847,高露潔 360度抗敏專家牙刷(1入),,高露潔 360度抗敏專家牙刷
108848,高露潔專業型超小刷頭密植軟毛牙刷6支裝【康是美】,,高露潔專業型超小刷頭密植軟毛牙刷康是美
108849,熊寶貝 沁藍海洋香柔軟衣物護衣精2+6組 (3.2lx2+1.84lx6包),,熊寶貝 沁藍海洋香柔軟衣物護衣精2


In [19]:
testdata.to_csv('cleaned_test.csv')